In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import scipy.spatial.distance as dist

import matplotlib as mpl

import iblofunmatch.inter as ibfm
output_dir = "output" 
plots_dir = "plots/stability/"

import os 
os.makedirs(output_dir, exist_ok=True)
os.makedirs(plots_dir, exist_ok=True)

Load dataset $Z$ and subset indices. Together with modified dataset $Z'$. 

In [ ]:
Z = np.loadtxt("Z_example.txt", dtype="float")
X_indices = np.loadtxt("X_idx_example.txt", dtype="int")
X = Z[X_indices]
# RandGen = np.random.default_rng(5)
# epsilon = 0.25
# noise = RandGen.uniform(0,epsilon,2 * Z.shape[0]).reshape((Z.shape[0],2))
# Zm = Z + noise
# np.savetxt("Zm_example.txt", Zm, fmt="%.4f")
Zm = np.loadtxt("Zm_example.txt", dtype="float")
print(Zm.shape)
Xm = Zm[X_indices]

In [ ]:
fig, ax = plt.subplots(ncols=1, figsize=(3,3))
#Scatter points and their modified versions
ax.scatter(X[:,0], X[:,1], color=mpl.colormaps["RdBu"](0.3/1.3), s=20, marker="o", zorder=2)
ax.scatter(Z[:,0], Z[:,1], color=mpl.colormaps["RdBu"](0.3/1.3), s=15, marker="x", zorder=2)
ax.scatter(Xm[:,0], Xm[:,1], color=mpl.colormaps["RdBu"](1/1.3), s=20, marker="o", zorder=2)
ax.scatter(Zm[:,0], Zm[:,1], color=mpl.colormaps["RdBu"](1/1.3), s=15, marker="x", zorder=2)
# Draw connecting lines between points 
xcoord = list(zip(Z[:,0], Zm[:,0]))
ycoord = list(zip(Z[:,1], Zm[:,1]))
for xline, yline in zip(xcoord, ycoord):
    ax.plot(xline, yline, c="black", zorder=1)
ax.set_axis_off()
plt.savefig(plots_dir + "points_noise_example.png")

Print Hausdorff distances $d_H(X, X')$ and $d_H(Z, Z')$

In [ ]:
d_H_X = np.max(np.sqrt(np.sum((X-Xm)**2, axis=1)))
d_H_Z = np.max(np.sqrt(np.sum((Z-Zm)**2, axis=1)))
print(f"d_H(X, X') = {d_H_X:.4f}")
print(f"d_H(Z, Z') = {d_H_Z:.4f}")

Now, we compute matchigns for the pairs $X \subset Z$ and $X' \subset Z'$. 

In [ ]:
match_XZ = ibfm.get_IBloFunMatch_subset(Z, Z, X_indices, output_dir, max_rad=-1, points=True, max_dim=1)
match_XZm = ibfm.get_IBloFunMatch_subset(Zm, Zm, X_indices, output_dir, max_rad=-1, points=True, max_dim=1)

In [ ]:
from importlib import reload
reload(ibfm)

In [ ]:
fig, ax = plt.subplots(figsize=(8,5))
ibfm.plot_bimodule_0_blofun(match_XZ, ax, fig, fill=False, linecolor=mpl.colormaps["RdBu"](1/1.3))
ibfm.plot_bimodule_0_blofun(match_XZm, ax, fig, fill=False, linecolor=mpl.colormaps["RdBu"](0.3/1.3))
# Plot matching lines between corners of finite rectangles
diagram_match = []
xcoord = np.hstack((
    np.array([match_XZ["S_barcode_0"][:,1]]).transpose(),
    np.array([match_XZm["S_barcode_0"][:,1]]).transpose()
))
ycoord = np.hstack((
    np.array([match_XZ["X_barcode_0"][match_XZ["induced_matching_0"]][:,1]]).transpose(),
    np.array([match_XZm["X_barcode_0"][match_XZ["induced_matching_0"]][:,1]]).transpose()
)).reshape(-1, 2)
for xline, yline in zip(xcoord, ycoord):
    ax.plot(xline, yline, c="black", zorder=0.2, linewidth=3)
# Connect horizontal infinity lines with vertical black marked lines 
unmatched_Z = [i for i in range(Z.shape[0]-1) if i not in match_XZ["induced_matching_0"]]
unmatched_Zm = [i for i in range(Zm.shape[0]-1) if i not in match_XZm["induced_matching_0"]]
y_unmatched_Z =  match_XZ["X_barcode_0"][unmatched_Z][:,1].tolist()
y_unmatched_Zm =  match_XZm["X_barcode_0"][unmatched_Zm][:,1].tolist()
y_unmatched_Z.sort()
y_unmatched_Zm.sort()
x = ax.get_xlim()[1]*0.9
for yline in zip(y_unmatched_Z, y_unmatched_Zm):
    ydiff = yline[1]-yline[0]
    ax.add_patch(mpl.patches.Rectangle([x, yline[0]], x, ydiff, facecolor="black", edgecolor="none", alpha=0.3, zorder=0.2))
plt.tight_layout()
plt.savefig(plots_dir + "matching_bimodules.png")

The matched rectangles and their differences are:

In [ ]:
from tabulate import tabulate
table = []
# Add matched rectangles to table
for xline, yline in zip(xcoord, ycoord):
    table.append([f"{xline[0]:6.3f}", f"{yline[0]:6.3f}", f"{xline[1]:6.3f}", f"{yline[1]:6.3f}", 
                  f"{np.abs(xline[1]-xline[0]):6.3f}", f"{np.abs(yline[1]-yline[0]):6.3f}"])

# Add infinity line matchings to table
unmatched_y = np.hstack((np.array([y_unmatched_Z]).transpose(), np.array([y_unmatched_Zm]).transpose()))
for yline in unmatched_y:
    table.append([f"{np.inf:6.3f}", f"{yline[0]:6.3f}", f"{np.inf:6.3f}", f"{yline[1]:6.3f}",
                  f"{np.inf-np.inf:6.3f}", f"{np.abs(yline[1]-yline[0]):6.3f}"])

print(tabulate(table, headers=["a", "b", "a'", "b'", "|a-a'|", "|b-b'|"]))